# Model evaluation using Sidekick
In this notebook you will learn how to use the Deployment API of the Peltarion platform via Sidekick to get predictions on samples and evaluate the performance of the deployed model in more detail.

Note: This notebook requires installation of Sidekick. To install the package within the notebook, run the following code:

`
import sys
!{sys.executable} -m pip install git+https://github.com/Peltarion/sidekick#egg=sidekick
`

For more information about Sidekick, see:
https://github.com/Peltarion/sidekick

In [ ]:
import os
import operator
import itertools
import resource
import zipfile

from IPython.display import display, Image
import pandas as pd
from PIL import Image
import sidekick

## Setup

### Path to preprocessed dataset

In [ ]:
zip_path = '/Users/joakim/Downloads/preprocessed.zip'
extract_path = '/Users/joakim/Downloads'

dataset_path = os.path.join(extract_path, 'preprocessed')

### Extract zip file

In [ ]:
zip_ref = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall(extract_path)
zip_ref.close()

### Platform deployment

In [ ]:
deploy_url = 'https:...'
deploy_token = '...'

### Helper functions

In [ ]:
def get_max_score(pred):
    max_key = 'None'
    max_score = 0
    dict = pred['class'].items()
    for key,score in dict:
        if score >= max_score:        
            max_key = key
            max_score = score
    return (max_key, max_score)

def get_image(path):
    im = Image.open(os.path.join(dataset_path, path))
    new_im = im.copy()
    new_im.format = 'jpeg'
    im.close()
    return new_im

## Getting single  predictions

### Deployment

In [ ]:
client = sidekick.Deployment(
    # Enter URL and token
    url=deploy_url,
    token=deploy_token
)

### Ground truth

In [ ]:
index_path = os.path.join(dataset_path, 'index.csv')
df = pd.read_csv(index_path)
df = df.sample(frac=1, random_state=2323)
df.head()

### Predict damage for one image

In [ ]:
im_path_list = iter(list(df['image']))

In [ ]:
im_path = next(im_path_list)
im = Image.open(os.path.join(dataset_path, im_path))
display(im)
pred = client.predict(image=im)
print(get_max_score(pred))

### Predict damage for multiple images

In [ ]:
first_rows = df.head()
for i, row in first_rows.iterrows():
    img = Image.open(os.path.join(dataset_path, row['image']))
    display(img)
    pred = client.predict(image=img)
    print('Ground truth: {}\nPrediction: {}'.format(row['class'], pred['class']))

## Getting predictions (batch)

### Filter out training data
The predictions on the evaluation subset will be used in the analysis of the deployed model.

In [ ]:
# Validation data
eval_df = df[df['subset']=='V'].copy()

### Batch request

In [ ]:
eval_df['image_url'] = eval_df['image']
eval_df['image'] = eval_df['image'].apply(lambda path: get_image(path))
predictions = client.predict_lazy(eval_df.to_dict('record'))
eval_df.head(1)

In [ ]:
#This may take several minutes...
preds = [p for p in predictions]
eval_df['pred'] = [p['class'] for p in preds]
eval_df.head(5)

## Evaluation

In [ ]:
dicts = eval_df['pred']
max_keys = []
max_scores = []

for i in dicts:
    max_val = max(i.items(), key=lambda k: k[1])     
    max_keys.append(max_val[0])
    max_scores.append(max_val[1])
eval_df['pred_class'] = max_keys
eval_df['pred_score'] = max_scores
eval_df.head(5)

### Worst misclassified examples

In [ ]:
wrong_df = eval_df.loc[eval_df['class'] != eval_df['pred_class']]
wrong_df = wrong_df.sort_values(by=['pred_score'], ascending=False)
first_rows = wrong_df.head(10)
for i, row in first_rows.iterrows():
    display(row['image'])
    print('Ground truth: {}, Prediction: {}, Score: {}'.format(row['class'], row['pred_class'], row['pred_score']))